In [3]:
import numpy as np 
import pandas as pd
import os
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import rbf_kernel
from itertools import combinations
from torch.utils.data import TensorDataset,DataLoader
import random

In [5]:
def get_binary_classification(X:np.ndarray,y:np.ndarray):
    return X,(y!=0).astype(int)
def sepearte_tasks(X:np.ndarray,y:np.ndarray):
    y_pos_mask = y == 1
    X_postive,y_positive  = X[y_pos_mask],y[y_pos_mask]
    X_negative,y_negative = X[~y_pos_mask],y[~y_pos_mask]
    return X_postive,y_positive,X_negative,y_negative

In [10]:
num_splits=12
skf = StratifiedKFold(n_splits=num_splits, shuffle=False)
def load_range_dataset_w_benign(data_name, start_month, end_month, year, folder='data/',verbose = False):
    
    if start_month != end_month:
        dataset_name = f'{year}-{start_month}to{year}-{end_month}'
    else:
        dataset_name = f'{year}-{start_month}'
    
    saved_data_file = os.path.join(folder, data_name, f'{dataset_name}_selected.npz')   

    data = np.load(saved_data_file, allow_pickle=True)
    if verbose:
        print(f"Loading data from {year}-{start_month} to {year}-{end_month}")
        print(folder)
        print(saved_data_file)
    X_train, y_train = data['X_train'], data['y_train']
    y_mal_family = data['y_mal_family']

    return X_train, y_train, y_mal_family
def load_train_api_graph(train_start,test_end,verbose = False):
    tasks = []
    data = np.load("./gen_apigraph_drebin/2012-01to2012-12_selected.npz")
    X,y = data["X_train"],data["y_train"]
    y_fam = y
    X,y = get_binary_classification(X,y)
    for _, test_index in skf.split(X, y):
        X_split, y_split = X[test_index], y_fam[test_index]
        tasks.append((X_split, y_split))
    for year in range(train_start,test_end+1):
        for month in range(1,13):
            month = f'{month:02}'
            X_train,y_train,y_train_family = load_range_dataset_w_benign(data_name="gen_apigraph_drebin/",start_month=month,end_month=month,year=year,folder="",verbose=True)
            tasks.append([X_train,y_train])
    # test_tasks =[]
    # ## For the test dataset
    # for year in range(test_start,test_end+1):
    #     for month in range(1,13):
    #         month = f'{month:02}'
    #         X_test,y_test,_ = load_range_dataset_w_benign(data_name="gen_apigraph_drebin/",start_month=month,end_month=month,year=year,folder="") 
    #         # X_test,y_test = get_binary_classification(X_test,y_test)
    #         test_tasks.append([torch.tensor(X_test,dtype=torch.float32),torch.tensor(y_test,dtype=torch.long)])
    return tasks


In [11]:
tasks_api = load_train_api_graph(2013,2018,verbose=True)

Loading data from 2013-01 to 2013-01

gen_apigraph_drebin/2013-01_selected.npz
Loading data from 2013-02 to 2013-02

gen_apigraph_drebin/2013-02_selected.npz
Loading data from 2013-03 to 2013-03

gen_apigraph_drebin/2013-03_selected.npz
Loading data from 2013-04 to 2013-04

gen_apigraph_drebin/2013-04_selected.npz
Loading data from 2013-05 to 2013-05

gen_apigraph_drebin/2013-05_selected.npz
Loading data from 2013-06 to 2013-06

gen_apigraph_drebin/2013-06_selected.npz
Loading data from 2013-07 to 2013-07

gen_apigraph_drebin/2013-07_selected.npz
Loading data from 2013-08 to 2013-08

gen_apigraph_drebin/2013-08_selected.npz
Loading data from 2013-09 to 2013-09

gen_apigraph_drebin/2013-09_selected.npz
Loading data from 2013-10 to 2013-10

gen_apigraph_drebin/2013-10_selected.npz
Loading data from 2013-11 to 2013-11

gen_apigraph_drebin/2013-11_selected.npz
Loading data from 2013-12 to 2013-12

gen_apigraph_drebin/2013-12_selected.npz
Loading data from 2014-01 to 2014-01

gen_apigraph_d

In [ ]:
for j in range(len(tasks_api)):
    X, y = tasks_api[j]
    # Get indices of non-zero labels
    pos_indices = np.where(y != 0)[0]  # Access the indices directly
    # Get binary classification
    X_np, y_np = get_binary_classification(X, y)
    # Separate tasks into positive and negative
    X_pos, y_pos, X_neg, y_neg = sepearte_tasks(X_np, y_np) 
    # Append metadata to positive and negative samples
    X_pos = [np.append(data, 90 + j) for data in X_pos]
    X_neg = [np.append(data, j) for data in X_neg]
    np.save(f"data_processed/test/api_graph/{90+j}.npy", X_pos)
    np.save(f"data_processed/test/api_graph/{90+j}_labels.npy", y[pos_indices])
    np.save(f"data_processed/test/api_graph/{j}.npy", X_neg)

In [12]:
def load_androzoo_derbin():
    first_year_data_file = os.path.join("gen_androzoo_drebin","2019-01to2019-12_selected.npz")
    first_year_data = np.load(first_year_data_file,allow_pickle=True)
    X,y = first_year_data["X_train"],first_year_data["y_train"]
    y_fam = y 
    X,y = get_binary_classification(X,y)
    tasks = []
    for _, test_index in skf.split(X, y):
        X_split, y_split = X[test_index], y_fam[test_index]
        tasks.append((X_split, y_split))
    for year in range(2020,2022):
            for month in range(1,13):
                month = f'{month:02}'
                X_train,y_train,_ = load_range_dataset_w_benign(data_name="gen_androzoo_drebin/",start_month=month,end_month=month,year=year,folder="")
                # X_train,y_train = get_binary_classification(X_train,y_train)
                tasks.append((X_train,y_train))
    return tasks

In [13]:
tasks_zoo = load_androzoo_derbin()

In [ ]:
for j in range(len(tasks_zoo)):
    X, y = tasks_zoo[j]
    # Get indices of non-zero labels
    pos_indices = np.where(y != 0)[0]  # Access the indices directly
    # Get binary classification
    X_np, y_np = get_binary_classification(X, y)
    # Separate tasks into positive and negative
    X_pos, y_pos, X_neg, y_neg = sepearte_tasks(X_np, y_np) 
    # Append metadata to positive and negative samples
    X_pos = [np.append(data, 90 + j) for data in X_pos]
    X_neg = [np.append(data, j) for data in X_neg]
    np.save(f"data_processed/androzoo/{90+j}.npy", X_pos)
    np.save(f"data_processed/androzoo/{90+j}_labels.npy", y[pos_indices])
    np.save(f"data_processed/androzoo/{j}.npy", X_neg)

In [ ]:
from sklearn.preprocessing import normalize
num_splits=12
skf = StratifiedKFold(n_splits=num_splits, shuffle=False)
def load_bodmas(folder, dataset):
    # Load the .npz file
    saved_data_file = os.path.join(folder, dataset)
    data = np.load(saved_data_file, allow_pickle=True)
    X, y = data['X'], data['y']
    df = pd.read_csv("./BODMAS/bodmas_metadata.csv")
    df['family_encoded'], unique_values =pd.factorize(df['family'].fillna("benign"))
    y_family = np.array(df['family_encoded'])
    # Convert to PyTorch tensors
    negative_indices = np.where(y == 0)[0] # Selecting the Benign Data
    positive_indices = np.where(y !=0)[0] # Select the Attack 
    random_positive_indices = random.sample(positive_indices.tolist(),int(10/90*len(negative_indices)))
    # print(len(random_positive_indices))
    y_pos_neg_mask = np.concatenate([negative_indices,positive_indices])
    y_pos_neg_mask.sort()
    X, y ,y_family= X[y_pos_neg_mask], y[y_pos_neg_mask] ,y_family [y_pos_neg_mask]
    X = normalize(X)
    tasks = []
    num_folds = 12
    fold_size = len(X) // num_folds
    tasks = []

    for _, test_index in skf.split(X, y):
        X_split, y_split,y_family_split = X[test_index], y[test_index],y_family[test_index]
        tasks.append((X_split, y_split, y_family_split))
    return tasks

In [ ]:
tasks_bod  = load_bodmas("./BODMAS/","bodmas.npz")

In [ ]:

for j in range(len(tasks_bod)):
    X,y ,y_family= tasks_bod[j]
    print(f"Shape of X: {X.shape}, y :{y.shape} and y_family: {y_family.shape}")
    X_np,y_np = get_binary_classification(X,y)
    X_pos,y_pos,X_neg,y_neg = sepearte_tasks(X_np,y_np)
    pos_indices = np.where(y != 0)[0]  # Access the indices directly
    X_pos = [np.append(data,90+j) for data in X_pos]
    X_neg = [np.append(data,j) for data in X_neg]
    # print(np.unique(y_family[pos_indices],return_counts=True) )
    print(f"Shape of X: {len(X_pos)},  and y_family: {len(y_family[pos_indices])}")
    np.save(f"data_processed/test/bodmas/{90+j}.npy",X_pos)
    np.save(f"data_processed/test/bodmas/{90+j}_labels.npy", y[pos_indices])
    np.save(f"data_processed/test/bodmas/{j}.npy",X_neg)